In [4]:
import cv2
import numpy as np

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import Model

from keras.preprocessing import image
import matplotlib.pyplot as plt

labels = [x for x in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ']
labels.append('del')
labels.append('nothing')
labels.append('space')
labels = sorted(labels)

In [10]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
pre_trained_model = MobileNetV2(input_shape=(192, 192, 3),
                               include_top=False,
                               weights='imagenet')
for layer in pre_trained_model.layers:
    layer.trainable = False
    
last_layer = pre_trained_model.get_layer('block_12_depthwise_relu')
last_output = last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(29, activation='softmax')(x)


model = Model(pre_trained_model.input, x)

model.load_weights('signlanguage_90_weights.h5')

model.compile(optimizer=RMSprop(lr=0.0001),
             loss='categorical_crossentropy',
             metrics=['acc'])

In [11]:
camera = cv2.VideoCapture(0)

top, right, bottom, left = 80, 350, 305, 590

while(True):
    (grabbed, frame) = camera.read()

    frame = cv2.flip(frame, 1)

    copy = frame.copy()

    (height, width) = frame.shape[:2]

    rect = frame[top:bottom, right:left]

    img = cv2.resize(rect, (192, 192))
    x = image.img_to_array(img) / 255.0
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    pred = model.predict(images)
    res = labels[np.argmax(pred)]
    
    cv2.rectangle(copy, (left, top), (right, bottom), (0,255,0), 2)

    font = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,30)
    fontScale = 1
    fontColor = (255,255,255)
    lineType = 2

    cv2.putText(copy, 'Model output: ' + str(res), 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)
    
    cv2.imshow("Camera", copy)

    # observe the keypress by the user
    keypress = cv2.waitKey(1) & 0xFF

    # if the user pressed "q", then stop looping
    if keypress == ord("q"):
        break

camera.release()
cv2.destroyAllWindows()

In [ ]:
camera.release()
cv2.destroyAllWindows()